In [1]:
import pandas as pd #Для работы с таблицами данных (дата фреймы)
import requests #Для запросов к серверу
import json #Для обработки ответов сервера

In [8]:
# В базовом запросе мы укажем:
# engines/stok (фондовый рынок),
# markets/shares - рынок акций,
# boards/TQBR - режим торгов для акции сбербанка TQBR

# Базовый запрос поместим в переменную base_url
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
response = requests.get(base_url) # Получим ответ от сервера 
result = json.loads(response.text) # Расшифровываем ответ от сервера

In [14]:
response.text

'{\n"history": {\n\t"metadata": {\n\t\t"BOARDID": {"type": "string", "bytes": 12, "max_size": 0},\n\t\t"TRADEDATE": {"type": "date", "bytes": 10, "max_size": 0},\n\t\t"SHORTNAME": {"type": "string", "bytes": 189, "max_size": 0},\n\t\t"SECID": {"type": "string", "bytes": 36, "max_size": 0},\n\t\t"NUMTRADES": {"type": "double"},\n\t\t"VALUE": {"type": "double"},\n\t\t"OPEN": {"type": "double"},\n\t\t"LOW": {"type": "double"},\n\t\t"HIGH": {"type": "double"},\n\t\t"LEGALCLOSEPRICE": {"type": "double"},\n\t\t"WAPRICE": {"type": "double"},\n\t\t"CLOSE": {"type": "double"},\n\t\t"VOLUME": {"type": "double"},\n\t\t"MARKETPRICE2": {"type": "double"},\n\t\t"MARKETPRICE3": {"type": "double"},\n\t\t"ADMITTEDQUOTE": {"type": "double"},\n\t\t"MP2VALTRD": {"type": "double"},\n\t\t"MARKETPRICE3TRADESVALUE": {"type": "double"},\n\t\t"ADMITTEDVALUE": {"type": "double"},\n\t\t"WAVAL": {"type": "double"},\n\t\t"TRADINGSESSION": {"type": "int32"},\n\t\t"CURRENCYID": {"type": "string", "bytes": 9, "max_siz

In [ ]:
col_name = result['history']['columns'] # Задаем имена колонок извлекая данные из ответа сервера
data_shares = pd.DataFrame(columns = col_name) #

In [3]:
data_shares  # Можно посмотреть что получилось

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR


In [15]:
url_share = 'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/SBER.json' #Первая страница данных
# Получаем данные
response = requests.get(url_share)
result = json.loads(response.text) 
resp_date = result['history']['data'] 
#Извлеченные данные добавляем в начало дата фрейма
data_shares  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

In [5]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = url_share + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_shares = pd.concat([data_shares, data_next_page], ignore_index=True) 
    a = len(resp_date)
    b = b + 100

In [6]:
b

2800

In [7]:
data_shares

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,TQBR,2013-03-25,Сбербанк,SBER,140,5.934000e+07,96.00,96.00,101.14,98.66,...,99.99,99.99,99.99,8.572434e+09,8.572434e+09,8572433621.95,NaN,3,SUR,NaN
1,TQBR,2013-03-26,Сбербанк,SBER,183,1.260304e+08,98.58,97.08,99.31,97.13,...,97.82,97.82,97.82,1.221847e+10,1.221847e+10,12218468579.299999,NaN,3,SUR,-1.61
2,TQBR,2013-03-27,Сбербанк,SBER,176,1.218359e+08,97.90,95.39,98.00,96.73,...,96.45,96.45,96.45,1.251981e+10,1.251981e+10,12519812986.549999,NaN,3,SUR,-0.46
3,TQBR,2013-03-28,Сбербанк,SBER,299,1.924698e+08,96.38,95.72,98.66,98.64,...,97.74,97.74,97.74,1.257281e+10,1.257281e+10,12572809760.42,NaN,3,SUR,1.90
4,TQBR,2013-03-29,Сбербанк,SBER,121,7.726886e+07,98.60,98.32,99.09,98.83,...,98.77,98.77,98.77,2.611474e+09,2.611474e+09,2611473832.5,NaN,3,SUR,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,TQBR,2024-04-17,Сбербанк,SBER,59080,8.679486e+09,308.65,305.91,309.74,306.43,...,307.56,307.56,None,8.023361e+09,8.023361e+09,None,0.0,3,SUR,-0.55
2792,TQBR,2024-04-18,Сбербанк,SBER,37902,6.264316e+09,306.01,305.05,308.29,308.16,...,307.19,307.19,None,5.717134e+09,5.717134e+09,None,0.0,3,SUR,0.46
2793,TQBR,2024-04-19,Сбербанк,SBER,38080,4.482221e+09,308.00,306.80,308.38,307.90,...,307.41,307.41,None,3.774031e+09,3.774031e+09,None,0.0,3,SUR,-0.20
2794,TQBR,2024-04-22,Сбербанк,SBER,138656,2.150335e+10,308.00,307.38,315.00,313.30,...,310.90,310.90,None,1.653942e+10,1.653942e+10,None,0.0,3,SUR,2.48
